# Building Your Predictor

The next step after preparing and importing your data via `Getting_Data_Ready.ipynb` is to build your first model.

The overall process for this is:

* Setup
* Create a Predictor
* Deploy a Predictor
* Obtain a Forecast

To get started, simply execute the cells below:


## Setup


Import the standard Python Libraries that are used in this lesson.

In [1]:
import sys
import os
import time
import boto3

# importing forecast notebook utility from notebooks/common directory
sys.path.insert( 0, os.path.abspath("../../common") )
import util

The line below will retrieve your shared variables from the first notebook.

In [4]:
%store -r

In [6]:
print(datasetGroupArn)
print(datasetArn)
print(rolearn)
print(key)
print(bucket_name)
print(region)
print(ds_import_job_arn)

arn:aws:forecast:ap-northeast-2:414501109622:dataset-group/electricity_ds_group3
arn:aws:forecast:ap-northeast-2:414501109622:dataset/electricity_demand_ds3
arn:aws:iam::414501109622:role/ForecastRole
elec_data/item-demand-time-train.csv
randombucketname
ap-northeast-2
arn:aws:forecast:ap-northeast-2:414501109622:dataset-import-job/electricity_demand_ds3/electricity_ds_import_job3


The last part of the setup process is to validate that your account can communicate with Amazon Forecast, the cell below does just that.

In [7]:
session = boto3.Session(region_name=region) 
forecast = session.client(service_name='forecast') 
forecastquery = session.client(service_name='forecastquery')

## Create a Predictor

Now in the previous notebook, your data was imported to be used by Forecast, here we will once again define your dataset information and then start building your model or predictor.

Forecast horizon is the number of number of time points to predicted in the future. For weekly data, a value of 12 means 12 weeks. Our example is hourly data, we try forecast the next day, so we can set to 24.

In [ ]:
predictorName= project+'_deeparp_algo'

In [9]:
forecastHorizon = 24

In [12]:
algorithmArn = 'arn:aws:forecast:::algorithm/Deep_AR_Plus'

In [13]:
FeaturizationConfig_dict = \
{"ForecastFrequency": "H", 
 "Featurizations":    [ {"AttributeName": "target_value", 
                         "FeaturizationPipeline": [ 
                             {"FeaturizationMethodName": "filling", 
                              "FeaturizationMethodParameters": {"frontfill": "none", 
                                                                "middlefill": "zero", 
                                                                "backfill": "zero"}}
                         ]
                        }
                      ]
}

In [14]:
create_predictor_response = \
    forecast.create_predictor(\
        PredictorName = "electricitypredictor3", 
        AlgorithmArn = algorithmArn,
        InputDataConfig = {"DatasetGroupArn": datasetGroupArn},
        ForecastHorizon = forecastHorizon,
        FeaturizationConfig = FeaturizationConfig_dict,
        PerformAutoML= False,
        PerformHPO=False,
        EvaluationParameters= {"NumberOfBacktestWindows": 1, 
                               "BackTestWindowOffset": 24}, 
         )

In [15]:
create_predictor_response

{'PredictorArn': 'arn:aws:forecast:ap-northeast-2:414501109622:predictor/electricitypredictor3',
 'ResponseMetadata': {'RequestId': '3c5ca8b0-8126-4453-9365-d2a639558aec',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Wed, 01 Apr 2020 07:49:16 GMT',
   'x-amzn-requestid': '3c5ca8b0-8126-4453-9365-d2a639558aec',
   'content-length': '95',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

In [16]:
predictor_arn=create_predictor_response['PredictorArn']
predictor_arn

'arn:aws:forecast:ap-northeast-2:414501109622:predictor/electricitypredictor3'

Check the status of the predictor. When the status change from **CREATE_IN_PROGRESS** to **ACTIVE**, we can continue to next steps. Depending on data size, model selection and hyper parameters，it can take 10 mins to more than one hour to be **ACTIVE**.

In [17]:
status_indicator = util.StatusIndicator()

while True:
    status = forecast.describe_predictor(PredictorArn=predictor_arn)['Status']
    status_indicator.update(status)
    if status in ('ACTIVE', 'CREATE_FAILED'): break
    time.sleep(10)

status_indicator.end()

CREATE_IN_PROGRESS ..........................................................................................................................................................
ACTIVE 


### Get Error Metrics

In [19]:
forecast.get_accuracy_metrics(PredictorArn=predictor_arn)

{'PredictorEvaluationResults': [{'AlgorithmArn': 'arn:aws:forecast:::algorithm/Deep_AR_Plus',
   'TestWindows': [{'EvaluationType': 'SUMMARY',
     'Metrics': {'RMSE': 7.132759657262377,
      'WeightedQuantileLosses': [{'Quantile': 0.9,
        'LossValue': 0.04120108962382528},
       {'Quantile': 0.5, 'LossValue': 0.08554207045872382},
       {'Quantile': 0.1, 'LossValue': 0.03793336116491365}]}},
    {'TestWindowStart': datetime.datetime(2014, 10, 30, 9, 0, tzinfo=tzlocal()),
     'TestWindowEnd': datetime.datetime(2014, 10, 31, 9, 0, tzinfo=tzlocal()),
     'ItemCount': 3,
     'EvaluationType': 'COMPUTED',
     'Metrics': {'RMSE': 7.132759657262378,
      'WeightedQuantileLosses': [{'Quantile': 0.9,
        'LossValue': 0.041201089623825286},
       {'Quantile': 0.5, 'LossValue': 0.08554207045872382},
       {'Quantile': 0.1, 'LossValue': 0.03793336116491365}]}}]}],
 'ResponseMetadata': {'RequestId': '195c58ee-8f7d-4f65-bd94-7af3e57ff24a',
  'HTTPStatusCode': 200,
  'HTTPHeaders'

## Create a Forecast

Now create a forecast using the model that was trained

In [21]:
forecastName= project+'_deeparp_algo_forecast'

In [22]:
create_forecast_response = \
    forecast.create_forecast(ForecastName=forecastName,
                             PredictorArn=predictor_arn)
forecast_arn = create_forecast_response['ForecastArn']

Check the status of the forecast process, when the status change from **CREATE_IN_PROGRESS** to **ACTIVE**, we can continue to next steps. Depending on data size, model selection and hyper parameters，it can take 10 mins to more than one hour to be **ACTIVE**.

In [23]:
status_indicator = util.StatusIndicator()

while True:
    status = forecast.describe_forecast(ForecastArn=forecast_arn)['Status']
    status_indicator.update(status)
    if status in ('ACTIVE', 'CREATE_FAILED'): break
    time.sleep(10)

status_indicator.end()

CREATE_PENDING ..
CREATE_IN_PROGRESS .................................................................................................................................................................................
ACTIVE 


### Get Forecast

Once created, the forecast results are ready and you view them. 

In [24]:
print(forecast_arn)
print()
forecastResponse = forecastquery.query_forecast(
    ForecastArn=forecast_arn,
    Filters={"item_id":"client_12"}
)
print(forecastResponse)

arn:aws:forecast:ap-northeast-2:414501109622:forecast/util_power_forecastdemo_deeparp_algo_forecast

{'Forecast': {'Predictions': {'p10': [{'Timestamp': '2014-10-31T00:00:00', 'Value': 40.94285583496094}, {'Timestamp': '2014-10-31T01:00:00', 'Value': 38.1968879699707}, {'Timestamp': '2014-10-31T02:00:00', 'Value': 37.97128677368164}, {'Timestamp': '2014-10-31T03:00:00', 'Value': 30.86043930053711}, {'Timestamp': '2014-10-31T04:00:00', 'Value': 28.626176834106445}, {'Timestamp': '2014-10-31T05:00:00', 'Value': 26.198205947875977}, {'Timestamp': '2014-10-31T06:00:00', 'Value': 28.212310791015625}, {'Timestamp': '2014-10-31T07:00:00', 'Value': 31.498701095581055}, {'Timestamp': '2014-10-31T08:00:00', 'Value': 45.15296936035156}, {'Timestamp': '2014-10-31T09:00:00', 'Value': 112.49766540527344}, {'Timestamp': '2014-10-31T10:00:00', 'Value': 125.12451934814453}, {'Timestamp': '2014-10-31T11:00:00', 'Value': 116.31916809082031}, {'Timestamp': '2014-10-31T12:00:00', 'Value': 126.6270141601562

## Next Steps

Now that your forecast has been created, to evaluate it use `3.Evaluating_Your_Predictor.ipynb` but before opening it, execute the cell below to share your variables again with the next notebook.

In [25]:
%store forecast_arn
%store predictor_arn

Stored 'forecast_arn' (str)
Stored 'predictor_arn' (str)
